In [1]:
## imporrt libarary 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [2]:
new_cust = pd.DataFrame({
    'Agency': ['EPX','CWT'],
    'Agency Type': ['Travel Agency','Travel Agency'], 
    'Distribution Channel': ['Online','Online'], 
    'Product Name': ['2 way Comprehensive Plan','Rental Vehicle Excess Insurance'],
    'Gender': [np.nan,'Not Specified'],  # Providing a placeholder value
    'Duration': [20,140], 
    'Destination': ['CHINA','SPAIN'], 
    'Net Sales': [30.0,0.0],
    'Commision (in value)': [0.00,23.76], 
    'Age': [36,31]
})
new_cust

,Agency,Agency Type,Distribution Channel,Product Name,Gender,Duration,Destination,Net Sales,Commision (in value),Age
0,EPX,Travel Agency,Online,2 way Comprehensive Plan,NaN,20,CHINA,30.0,0.00,36
1,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,Not Specified,140,SPAIN,0.0,23.76,31


In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/ivanrseptian/ivan-038/sa-development.json"

In [4]:
project_id = 'dti-ds'
dataset_id = 'ivan_dataset_038'
table_id = 'x_test_new'
region = 'us-central1'
bucket_name = 'ivan_gcs_038'
blob_name = 'data/X_test_new.csv'

In [5]:
# Upload the model to Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob('model/final_model_xgb.pkl')
    blob_model.download_to_filename('final_model_xgb.pkl')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [6]:
#load data from BQ
## using bigquery client 
# client = bigquery.Client(credentials=credentials,project=project_id)
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df = query_job.result().to_dataframe()
df.columns = df.columns.str.replace('_', ' ')
df.rename(columns={'Commision  in value ': 'Commision (in value)'}, inplace=True)

/home/ivanrseptian/miniconda3/envs/capstone3-env/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [7]:
df.columns

Index(['Agency', 'Agency Type', 'Distribution Channel', 'Product Name',
       'Gender', 'Duration', 'Destination', 'Net Sales',
       'Commision (in value)', 'Age'],
      dtype='object')

In [8]:
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, RobustScaler
import pickle
continent_map = {
    'SINGAPORE': 'Asia',
    'MALAYSIA': 'Asia',
    'INDIA': 'Asia',
    'UNITED STATES': 'North America',
    'KOREA, REPUBLIC OF': 'Asia',
    'THAILAND': 'Asia',
    'GERMANY': 'Europe',
    'JAPAN': 'Asia',
    'INDONESIA': 'Asia',
    'VIET NAM': 'Asia',
    'AUSTRALIA': 'Oceania',
    'FINLAND': 'Europe',
    'UNITED KINGDOM': 'Europe',
    'SRI LANKA': 'Asia',
    'SPAIN': 'Europe',
    'HONG KONG': 'Asia',
    'MACAO': 'Asia',
    'CHINA': 'Asia',
    'UNITED ARAB EMIRATES': 'Asia',
    'IRAN, ISLAMIC REPUBLIC OF': 'Asia',
    'TAIWAN, PROVINCE OF CHINA': 'Asia',
    'POLAND': 'Europe',
    'CANADA': 'North America',
    'OMAN': 'Asia',
    'PHILIPPINES': 'Asia',
    'GREECE': 'Europe',
    'BELGIUM': 'Europe',
    'TURKEY': 'Asia',
    'BRUNEI DARUSSALAM': 'Asia',
    'DENMARK': 'Europe',
    'SWITZERLAND': 'Europe',
    'NETHERLANDS': 'Europe',
    'SWEDEN': 'Europe',
    'MYANMAR': 'Asia',
    'KENYA': 'Africa',
    'CZECH REPUBLIC': 'Europe',
    'FRANCE': 'Europe',
    'RUSSIAN FEDERATION': 'Europe',
    'PAKISTAN': 'Asia',
    'ARGENTINA': 'South America',
    'TANZANIA, UNITED REPUBLIC OF': 'Africa',
    'SERBIA': 'Europe',
    'ITALY': 'Europe',
    'CROATIA': 'Europe',
    'NEW ZEALAND': 'Oceania',
    'PERU': 'South America',
    'MONGOLIA': 'Asia',
    'CAMBODIA': 'Asia',
    'QATAR': 'Asia',
    'NORWAY': 'Europe',
    'LUXEMBOURG': 'Europe',
    'MALTA': 'Europe',
    "LAO PEOPLE'S DEMOCRATIC REPUBLIC": 'Asia',
    'ISRAEL': 'Asia',
    'SAUDI ARABIA': 'Asia',
    'AUSTRIA': 'Europe',
    'PORTUGAL': 'Europe',
    'NEPAL': 'Asia',
    'UKRAINE': 'Europe',
    'ESTONIA': 'Europe',
    'ICELAND': 'Europe',
    'BRAZIL': 'South America',
    'MEXICO': 'North America',
    'CAYMAN ISLANDS': 'North America',
    'PANAMA': 'North America',
    'BANGLADESH': 'Asia',
    'TURKMENISTAN': 'Asia',
    'BAHRAIN': 'Asia',
    'KAZAKHSTAN': 'Asia',
    'TUNISIA': 'Africa',
    'IRELAND': 'Europe',
    'ETHIOPIA': 'Africa',
    'NORTHERN MARIANA ISLANDS': 'Oceania',
    'MALDIVES': 'Asia',
    'SOUTH AFRICA': 'Africa',
    'VENEZUELA': 'South America',
    'COSTA RICA': 'North America',
    'JORDAN': 'Asia',
    'MALI': 'Africa',
    'CYPRUS': 'Europe',
    'MAURITIUS': 'Africa',
    'LEBANON': 'Asia',
    'KUWAIT': 'Asia',
    'AZERBAIJAN': 'Asia',
    'HUNGARY': 'Europe',
    'BHUTAN': 'Asia',
    'BELARUS': 'Europe',
    'MOROCCO': 'Africa',
    'ECUADOR': 'South America',
    'UZBEKISTAN': 'Asia',
    'CHILE': 'South America',
    'FIJI': 'Oceania',
    'PAPUA NEW GUINEA': 'Oceania',
    'ANGOLA': 'Africa',
    'FRENCH POLYNESIA': 'Oceania',
    'NIGERIA': 'Africa',
    'MACEDONIA, THE FORMER YUGOSLAV REPUBLIC OF': 'Europe',
    'NAMIBIA': 'Africa',
    'GEORGIA': 'Asia',
    'COLOMBIA': 'South America',
    'SLOVENIA': 'Europe',
    'EGYPT': 'Africa',
    'ZIMBABWE': 'Africa',
    'BULGARIA': 'Europe',
    'BERMUDA': 'North America',
    'URUGUAY': 'South America',
    'GUINEA': 'Africa',
    'GHANA': 'Africa',
    'BOLIVIA': 'South America',
    'TRINIDAD AND TOBAGO': 'North America',
    'VANUATU': 'Oceania',
    'GUAM': 'Oceania',
    'UGANDA': 'Africa',
    'JAMAICA': 'North America',
    'LATVIA': 'Europe',
    'ROMANIA': 'Europe',
    'REPUBLIC OF MONTENEGRO': 'Europe',
    'KYRGYZSTAN': 'Asia',
    'GUADELOUPE': 'North America',
    'ZAMBIA': 'Africa',
    'RWANDA': 'Africa',
    'BOTSWANA': 'Africa',
    'GUYANA': 'South America',
    'LITHUANIA': 'Europe',
    'GUINEA-BISSAU': 'Africa',
    'SENEGAL': 'Africa',
    'CAMEROON': 'Africa',
    'SAMOA': 'Oceania',
    'PUERTO RICO': 'North America',
    'TAJIKISTAN': 'Asia',
    'ARMENIA': 'Asia',
    'FAROE ISLANDS': 'Europe',
    'DOMINICAN REPUBLIC': 'North America',
    'MOLDOVA, REPUBLIC OF': 'Europe',
    'BENIN': 'Africa',
    'REUNION': 'Africa',
    "KOREA, DEMOCRATIC PEOPLE'S REPUBLIC OF": 'Asia',
    'TIBET': 'Asia'
}

mapping = {
    'Annual Silver Plan': 'Annual Plans',
    'Cancellation Plan': 'Specialized Plans',
    'Basic Plan': 'Basic Plans',
    '2 way Comprehensive Plan': 'Comprehensive Plans',
    'Bronze Plan': 'Other',
    '1 way Comprehensive Plan': 'Comprehensive Plans',
    'Rental Vehicle Excess Insurance': 'Specialized Plans',
    'Single Trip Travel Protect Gold': 'Other Plans',
    'Silver Plan': 'Other Plans',
    'Value Plan': 'Basic Plans',
    '24 Protect': 'Other Plans',
    'Annual Travel Protect Gold': 'Annual Plans',
    'Comprehensive Plan': 'Other',
    'Ticket Protector': 'Specialized Plans',
    'Travel Cruise Protect': 'Specialized Plans',
    'Single Trip Travel Protect Silver': 'Other Plans',
    'Individual Comprehensive Plan': 'Comprehensive Plans',
    'Gold Plan': 'Other Plans',
    'Annual Gold Plan': 'Annual Plans',
    'Child Comprehensive Plan': 'Comprehensive Plans',
    'Premier Plan': 'Basic Plans',
    'Annual Travel Protect Silver': 'Annual Plans',
    'Single Trip Travel Protect Platinum': 'Other Plans',
    'Annual Travel Protect Platinum': 'Annual Plans',
    'Spouse or Parents Comprehensive Plan': 'Other Plans',
    'Travel Cruise Protect Family': 'Specialized Plans'
}


# Define your custom functions and transformers
def add_new_column_continent(X):
    X = X.copy()
    X['Continent'] = X['Destination'].apply(lambda dest: continent_map.get(dest, 'Unknown'))
    return X

add_column_transformer_continent = FunctionTransformer(add_new_column_continent, validate=False)

def add_new_column_product_name_category(Y):
    Y = Y.copy()
    Y['Product Name Category'] = Y['Product Name'].apply(lambda name: mapping.get(name, 'Unknown'))
    return Y

add_column_transformer_category = FunctionTransformer(add_new_column_product_name_category, validate=False)




# load the model
with open('final_model_xgb.pkl','rb') as f:
    loaded_model = pickle.load(f)

y_pred_cloud = loaded_model.predict(df)
y_pred_cloud

array([1, 0, 1, ..., 0, 0, 0])

In [9]:
y_cloud = df.copy()
y_cloud['Pred Claim'] = y_pred_cloud
y_cloud.rename(columns={'Commision (in value)': 'Commision'}, inplace=True)

In [10]:
y_cloud.columns

Index(['Agency', 'Agency Type', 'Distribution Channel', 'Product Name',
       'Gender', 'Duration', 'Destination', 'Net Sales', 'Commision', 'Age',
       'Pred Claim'],
      dtype='object')

In [11]:
df.columns

Index(['Agency', 'Agency Type', 'Distribution Channel', 'Product Name',
       'Gender', 'Duration', 'Destination', 'Net Sales',
       'Commision (in value)', 'Age'],
      dtype='object')

In [12]:
df.rename(columns={'Commision_(in_value)': 'Commision'}, inplace=True)

In [13]:
y_pred_cloud_2 = loaded_model.predict(new_cust)
y_pred_cloud_2

array([1, 0])

In [14]:
y_cloud_2 = new_cust.copy()
y_cloud_2['Pred Claim'] = y_pred_cloud_2
y_cloud_2.rename(columns={'Commision (in value)': 'Commision'}, inplace=True)

In [15]:
y_cloud_2.columns

Index(['Agency', 'Agency Type', 'Distribution Channel', 'Product Name',
       'Gender', 'Duration', 'Destination', 'Net Sales', 'Commision', 'Age',
       'Pred Claim'],
      dtype='object')

In [16]:
from google.cloud import bigquery
table_id = 'new_table_predict_capstone'
client = bigquery.Client()
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

y_cloud.columns = ['_'.join(i.split(' '))for i in y_cloud.columns]
job = client.load_table_from_dataframe(y_cloud, table_full_id)

job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}") 

Loaded 8866 rows into dti-ds.ivan_dataset_038.new_table_predict_capstone
